In [ ]:
!pip install transformers datasets peft torch


#### Load the Baby LLM (BioBERT)

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
from peft import LoraConfig, get_peft_model

# Load BioBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = BertForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Check model architecture
model.config


#### Define PEFT config

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Dimension of the low-rank adaptation
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout probability for LoRA layers
    lora_mode="question-answering"  # Mode of operation for QA
)

# Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)


#### Dataset preparation

In [ ]:
from datasets import Dataset

# Example data with scores
data = [
    {"context": "Context 1", "question": "Question 1", "answer": "Answer 1", "score": 1.0},
    {"context": "Context 1", "question": "Question 1", "answer": "Answer 2", "score": 0.5},
    # Add more examples...
]

dataset = Dataset.from_list(data)

def tokenize_function(examples):
    return tokenizer(
        
        examples["question"], 
        truncation=True, 
        padding="max_length", 
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)



#### Train the PEFT Model using SFTTrainer

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()


#### Prepare eval data

In [ ]:
# Example evaluation data
eval_data = [
    {"context": "Context 1", "question": "Question 1", "answer": "Correct Answer 1"},
    {"context": "Context 2", "question": "Question 2", "answer": "Correct Answer 2"},
    # Add more examples...
]

eval_dataset = Dataset.from_list(eval_data)

# Tokenize the dataset
def tokenize_eval_function(examples):
    return tokenizer(
       
        examples["question"], 
        truncation=True, 
        padding="max_length", 
        return_tensors="pt"
    )

tokenized_eval_dataset = eval_dataset.map(tokenize_eval_function, batched=True)


#### Define eval metrices

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(predictions, references):
    exact_matches = sum([1 if p == r else 0 for p, r in zip(predictions, references)])
    em_score = exact_matches / len(predictions)
    
    f1 = f1_score(references, predictions, average='weighted')
    
    return {"exact_match": em_score, "f1": f1}


#### Evaluator function

In [ ]:
import torch

def evaluate_model(model, eval_dataset, tokenizer):
    model.eval()
    predictions = []
    references = []

    for example in eval_dataset:
        inputs = tokenizer(           
            example['question'],
            truncation=True,
            padding=True,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = model(**inputs)
            answer_start = torch.argmax(outputs.start_logits)
            answer_end = torch.argmax(outputs.end_logits) + 1
            predicted_answer = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end])
            )
        
        predictions.append(predicted_answer)
        references.append(example['answer'])
    
    return compute_metrics(predictions, references)



#### Evaluate the model

In [ ]:

metrics = evaluate_model(peft_model, tokenized_eval_dataset, tokenizer)
print("Evaluation Metrics:", metrics)
